In [11]:
!pip install eli5

     |████████████████████████████████| 112kB 9.8MB/s 


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import eli5
from nltk.tokenize import TweetTokenizer
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
pd.set_option('max_colwidth',400)
import msgpack
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
info = pd.read_csv('train_info.csv')
info.head()

,id,injection
0,14577,False
1,60186,True
2,58459,True
3,87091,True
4,44461,False


In [3]:
info.injection.value_counts()

False    33499
True     32355
Name: injection, dtype: int64

In [4]:
with open('train.msgpack','rb') as data_file:
    train=msgpack.unpack(data_file)

In [5]:
with open('test.msgpack','rb') as data_file:
    test = msgpack.unpack(data_file)

In [6]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)
train.columns = ['id', 'text']
test.columns = ['id', 'text']

### EDA 

In [7]:
train.head(5).values

array([[59290,
        "' and/**/38>( select\t(622)/**/from/*362 emhgpv gpnqdn odpxkn qgnoyb */htipaw)"],
       [54992,
        'shqpkt" union /*!426 all\t*/(select kwicwt(\t(\tyaaoda\t()), (236), (konjlq()),  619, nyknas ()), byamlo (), \txfsesf( (\tdqoyrp()), (sbecbl()), (\tlhnjpg()), (kkicjp()), (udkygv()), hoewxv\t()))'],
       [64287,
        'nhnqag" uniondistinct--154 298 plhlre exaloq unyote \r(select (rpfeir#15 256 15 rttlat 66 gsjxjq 44 819 \r()), kuoopt(), (#\rvxhuha), (116), (--kfbvww 193 \r"mkhkjq"), 271, (omqjmj), gvdira--\r((--\r325), --mcrgew oqrhwp jhseux 55 xpayss 88 \rfctnjs (), (794), 3, #58 14 114 vdqoyq \r\'apxkvx\', (\tmhjblm())), #yxnynl \r"lllpue"/**/from#\rexample.example)'],
       [28821, 'D8j+oNbylTIGw='],
       [27825,
        'ened \\">\\n  <head>\\n    <script data-react-helmet=\\"true\\" type=\\"text/javascript\\">(function(w...<some bytes>....getTime(),event:\'example.example\'});var f=d.getElementsByTag...<some bytes>...re(j,f);\\n    })(window,doc

### Preprocessing

In [8]:
train['text'] = train['text'].astype(str)
test['text'] = test['text'].astype(str)

In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char')
full_text = list(train['text'].values) + list(test['text'].values)
vectorizer.fit(full_text)
train_vectorized = vectorizer.transform(train['text'])
test_vectorized = vectorizer.transform(test['text'])

In [13]:
info.injection.value_counts()

False    33499
True     32355
Name: injection, dtype: int64

In [14]:
info = pd.merge(train,info,on='id')

In [15]:
y = np.array([1 if i == True else 0 for i in info.injection.values])

In [16]:
logistic_regressor = LogisticRegression(C=10.0)

In [17]:
scores = cross_val_score(logistic_regressor, train_vectorized, y, scoring='roc_auc', n_jobs=-1, cv=5)
print('Cross-validation mean auc {0:.2f}%, std {1:.2f}.'.format(np.mean(scores) * 100, np.std(scores) * 100))

Cross-validation mean auc 99.92%, std 0.02.


In [18]:
logistic_regressor.fit(train_vectorized,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
eli5.show_weights(logistic_regressor, vec=vectorizer,  targets=[0, 1])

Weight?,Feature
+17.202,)
+15.926,'
+15.531,()
+12.341,
+11.493,*
+9.395,(
+9.074,*/
+8.873,/
+8.503,=%
+8.488,/*


In [21]:
eli5.show_prediction(logistic_regressor, doc=train['text'].values[100], vec=vectorizer)

In [22]:
### K- Fold Classification

In [23]:
from sklearn.model_selection import StratifiedKFold, KFold
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

In [24]:
def train_model(X=train_vectorized, X_test=test_vectorized, y=y, params=None, folds=folds, model_type='lgb', plot_feature_importance=False):

    oof = np.zeros(X.shape[0])
    prediction = np.zeros(X_test.shape[0])
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
                    verbose=1000, early_stopping_rounds=200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))
        
        prediction += y_pred    

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

    return oof, prediction

In [25]:
import time
params = {'num_leaves': 54,
         'min_data_in_leaf': 79,
         'objective': 'regression',
         'max_depth': 7,
         'learning_rate': 0.018545526395058548,
         "boosting": "gbdt",
         "feature_fraction": 0.8354507676881442,
         "bagging_freq": 3,
         "bagging_fraction": 0.8126672064208567,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         'min_child_weight': 5.343384366323818,
         'reg_alpha': 1.1302650970728192,
         'reg_lambda': 0.3603427518866501,
         'subsample': 0.8767547959893627,}

oof_lgb, prediction_lgb = train_model(X=train_vectorized, X_test=test_vectorized, y=y, folds=folds,
                                                          params=params, model_type='lgb')

Fold 0 started at Mon May 17 15:38:29 2021
Training until validation scores don't improve for 200 rounds.


KeyboardInterrupt: ignored